In [ ]:
# %load init.py
import os
import pickle
import sys
# Enable module import from the parent directory from notebooks
sys.path.append(os.path.abspath('..'))
import time

import matplotlib as mpl
# Select plotting backend
mpl.use('nbAgg')

import matplotlib.pyplot as plt
# Customize plotting
plt.style.use('seaborn-paper')
plt.rcParams['axes.labelsize'] = 11.0
plt.rcParams['axes.titlesize'] = 12.0
plt.rcParams['errorbar.capsize'] = 3.0
plt.rcParams['figure.dpi'] = 72.0
plt.rcParams['figure.titlesize'] = 12.0
plt.rcParams['legend.fontsize'] = 10.
plt.rcParams['lines.linewidth'] = 1.
plt.rcParams['xtick.labelsize'] = 11.0
plt.rcParams['ytick.labelsize'] = 11.0

import numpy as np
import sympy as sp
sp.init_printing(euler=True, use_latex=True)

from IPython import display
from scipy import io, optimize
from sklearn import metrics

import core
import dynamicals
import kernels
import numericals
import utils

In [ ]:
dynamical = dynamicals.Lorenz96(500)

# Example
Run the inference algorithm

In [ ]:
config = core.Config()
config.create_time(0, 4, 80, 0, 4, 8, 0, 4, 8)
config.X_0 = np.random.random(dynamical.num_x) * 8.
config.theta = np.array([8.]) 
config.rho_2 = np.full(dynamical.num_x, 4.) 

config.phi = [
    # (Kernal name, Kernal parameters)
    ('rbf', np.array([4.2, 0.1]))
] * dynamical.num_x
config.sigma_2 = np.full(dynamical.num_x, 1.) 
config.delta = np.full(dynamical.num_x, True)
config.delta[np.random.permutation(dynamical.num_x)[:int(0.35 * dynamical.num_x)]] = False
config.gamma = np.full(dynamical.num_x, 5e-2) 
config.gamma[config.delta] = 1e-1

config.opt_method = 'Newton-CG'
config.opt_tol = 1e-6
config.max_init_iter = 10
config.max_iter = 1000

config.plotting_enabled = True
config.plotting_freq = 50

config.debug = False

config.spl_X = dynamical.generate_sample_path(config.theta, config.rho_2, config.X_0, config.spl_tps)
config.obs_Y = utils.collect_observations(config.spl_X, config.obs_t_indices, config.sigma_2)

gp = core.GaussianProcessRegression(dynamical, config)
gp.run()

lpmf = core.LaplaceMeanFieldSDE(dynamical, config, gp)
lpmf.run()

# Experiment
Setup for the experiment

In [ ]:
sde_directory = '../data/sde-lorenz-96-scalability/sde-x-{}/'
ode_directory = '../data/sde-lorenz-96-scalability/ode-x-{}/'

config_filename = utils.CONFIG_FILENAME
data_filename = utils.DATA_FILENAME

num_states = [25, 50, 100, 200, 400, 800]
num_repetitions = 10

Plot the runtime

In [ ]:
sde_runtime_mean = []
sde_runtime_var = []
ode_runtime_mean = []
ode_runtime_var = []

for num_state in num_states:
    # Load data
    config = core.Config()
    config.load_config(sde_directory.format(num_state), config_filename)
    
    # Load data from LPMF-SDE
    data = []
    for i in range(1, num_repetitions + 1):
        tmp = utils.load_data(sde_directory.format(num_state), data_filename.format(i))
        if not np.alltrue(tmp['eta_theta'] > 0):
            raise RuntimeError('Negative theta value encountered for rode {}'.format(i))
        data.append(tmp)
    
    sde_runtime_mean.append(utils.get_runtime_mean(data))
    sde_runtime_var.append(utils.get_runtime_var(data))

for num_state in num_states:
    # Load data
    config = core.Config()
    config.load_config(ode_directory.format(num_state), config_filename)
    
    # Load data from LPMF-SDE
    data = []
    for i in range(1, num_repetitions + 1):
        tmp = utils.load_data(ode_directory.format(num_state), data_filename.format(i))
        if not np.alltrue(tmp['eta_theta'] > 0):
            raise RuntimeError('Negative theta value encountered for rode {}'.format(i))
        data.append(tmp)
    
    ode_runtime_mean.append(utils.get_runtime_mean(data))
    ode_runtime_var.append(utils.get_runtime_var(data))

figure = plt.figure(figsize=plt.figaspect(0.4))
ax = plt.gca()
ax.errorbar(num_states, sde_runtime_mean, color='C0', linestyle='-', linewidth=1.5, label='LPMF-SDE', 
            yerr=np.sqrt(sde_runtime_var), ecolor='0', elinewidth=1., capsize=3., capthick=.5)
ax.errorbar(num_states, ode_runtime_mean, color='C1', linestyle='-.', linewidth=1.5, label='LPMF', 
            yerr=np.sqrt(ode_runtime_var), ecolor='0', elinewidth=1., capsize=3., capthick=.5)
ax.set_ylabel('Runtime (s)', fontsize=14)
ax.set_xlabel('Number of States', fontsize=14)
ax.set_xlim([0, 810])
ax.set_ylim([0, None])
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles, labels=labels, loc=0)
figure.tight_layout()
plt.show()
figure.savefig('lorenz-96-scalability.eps', format='eps', dpi=1000, bbox_inches='tight')

In [ ]:
# Helper to generate config files
sde_directory = '../data/sde-lorenz-96-scalability/sde-x-{}/'
ode_directory = '../data/sde-lorenz-96-scalability/ode-x-{}/'

for num_state in num_states:
    dynamical = dynamicals.Lorenz96(num_state)
    config = core.Config()
    config.create_time(0, 4, 80, 0, 4, 8, 0, 4, 8)
    config.X_0 = np.random.random(dynamical.num_x) * 8.
    config.theta = np.array([8.]) 
    config.rho_2 = np.full(dynamical.num_x, 4.) 

    config.phi = [
        # (Kernal name, Kernal parameters)
        ('rbf', np.array([4.2, 0.1]))
    ] * dynamical.num_x
    config.sigma_2 = np.full(dynamical.num_x, 1.) 
    config.delta = np.full(dynamical.num_x, True)
    config.delta[np.random.permutation(dynamical.num_x)[:int(0.35 * dynamical.num_x)]] = False
    config.gamma = np.full(dynamical.num_x, 5e-2) 
    config.gamma[config.delta] = 1e-1

    config.opt_method = 'Newton-CG'
    config.opt_tol = 1e-6
    config.max_init_iter = 10
    config.max_iter = 1000

    config.plotting_enabled = True
    config.plotting_freq = 50
    
    config.debug = False

    config.spl_X = dynamical.generate_sample_path(config.theta, config.rho_2, config.X_0, config.spl_tps)
    config.obs_Y = utils.collect_observations(config.spl_X, config.obs_t_indices, config.sigma_2)
    
    config.save_config(sde_directory.format(num_state), config_filename)
    
    config.rho_2 = None
    config.save_config(ode_directory.format(num_state), config_filename)